In [2]:
import importlib

from pymavlink import mavutil
# import utils_MAV

import time
import sys

# importlib.reload(utils_MAV)

# BASE SUR LE SITE
# https://www.ardusub.com/developers/pymavlink.html#send-rc-joystick

# CONNECTION

In [3]:
# master = mavutil.mavlink_connection('/dev/ttyUSB2', baud=57600)  # Adjust IP and port if needed
# master = mavutil.mavlink_connection('/dev/ttyACM0', baud=115200)
# master = mavutil.mavlink_connection('udp:127.0.0.1:14541')
# master = mavutil.mavlink_connection('udp:127.0.0.1:14032', source_system=1, source_component=2)
master = mavutil.mavlink_connection('tcp:127.0.0.1:5783', baud=115200)
# self.mav_master = mavutil.mavlink_connection('/dev/ttyACM0', baud=115200)

print("Waiting for heartbeat...")
master.wait_heartbeat()
print("Heartbeat received! System ready.")

Waiting for heartbeat...
Heartbeat received! System ready.


In [45]:
# Disconnect
print("Disconnecting...")
master.close()
print("Connection closed.")

Disconnecting...
Connection closed.


In [35]:
for i in range(255):
    for j in range(255):
        master.mav.command_long_send(
            i, j,
            mavutil.mavlink.MAV_CMD_SET_MESSAGE_INTERVAL, 0,
            245, 1_000_000, 0, 0, 0, 0, 2)

In [48]:
master.mav.command_long_send(
            master.target_system, master.target_component,
            mavutil.mavlink.MAV_CMD_SET_MESSAGE_INTERVAL, 0,
            245, 1_000_000, 0, 0, 0, 0, 0)
master.mav.command_long_send(
            master.target_system, master.target_component,
            mavutil.mavlink.MAV_CMD_SET_MESSAGE_INTERVAL, 0,
            253, 500_000, 0, 0, 0, 0, 0)

In [27]:
print("Heartbeat from system (system %u component %u)" % (master.target_system, master.target_component))

Heartbeat from system (system 1 component 0)


In [31]:
from pymavlink import mavutil

# Start a connection listening on a UDP port
connection = master

# Wait for the first heartbeat to set the system and component ID of remote system for the link
connection.wait_heartbeat()
print("Heartbeat from system (system %u component %u)" % (connection.target_system, connection.target_component))

# Define command_long_encode message to send MAV_CMD_SET_MESSAGE_INTERVAL command
# param1: MAVLINK_MSG_ID_BATTERY_STATUS (message to stream)
# param2: 1000000 (Stream interval in microseconds)
message = connection.mav.command_long_encode(
        connection.target_system,  # Target system ID
        connection.target_component,  # Target component ID
        mavutil.mavlink.MAV_CMD_SET_MESSAGE_INTERVAL,  # ID of command to send
        0,  # Confirmation
        mavutil.mavlink.MAVLINK_MSG_ID_BATTERY_STATUS,  # param1: Message ID to be streamed
        10000, # param2: Interval in microseconds
        0,       # param3 (unused)
        0,       # param4 (unused)
        0,       # param5 (unused)
        0,       # param5 (unused)
        0        # param6 (unused)
        )

# Send the COMMAND_LONG
connection.mav.send(message)

# Wait for a response (blocking) to the MAV_CMD_SET_MESSAGE_INTERVAL command and print result
response = connection.recv_match(type='COMMAND_ACK', blocking=True)
if response and response.command == mavutil.mavlink.MAV_CMD_SET_MESSAGE_INTERVAL and response.result == mavutil.mavlink.MAV_RESULT_ACCEPTED:
    print("Command accepted")
else:
    print("Command failed")

Heartbeat from system (system 1 component 0)
Command accepted


# SEND ORDERS

In [4]:
for i in range(1000):
    try:
        print(master.recv_match().to_dict())
    except:
        pass
    time.sleep(0.1)

{'mavpackettype': 'HEARTBEAT', 'type': 13, 'autopilot': 3, 'base_mode': 217, 'custom_mode': 4, 'system_status': 4, 'mavlink_version': 3}
{'mavpackettype': 'SYS_STATUS', 'onboard_control_sensors_present': 1399979055, 'onboard_control_sensors_enabled': 1398930479, 'onboard_control_sensors_health': 1467087919, 'load': 0, 'voltage_battery': 12600, 'current_battery': 2809, 'battery_remaining': 0, 'drop_rate_comm': 0, 'errors_comm': 0, 'errors_count1': 0, 'errors_count2': 0, 'errors_count3': 0, 'errors_count4': 0}
{'mavpackettype': 'GPS_GLOBAL_ORIGIN', 'latitude': 486127587, 'longitude': 23963256, 'altitude': 100, 'time_usec': 2210639557}
{'mavpackettype': 'HOME_POSITION', 'latitude': 486127584, 'longitude': 23963255, 'altitude': 70, 'x': -0.03339565545320511, 'y': -0.007359788287431002, 'z': 0.029999999329447746, 'q': [nan, nan, nan, nan], 'approach_x': 0.0, 'approach_y': 0.0, 'approach_z': 0.0, 'time_usec': 2210639557}
{'mavpackettype': 'HEARTBEAT', 'type': 13, 'autopilot': 3, 'base_mode':

In [5]:
importlib.reload(utils_MAV)
utils_MAV.mav_arm(master)

True

In [6]:
importlib.reload(utils_MAV)
utils_MAV.mav_disarm(master, True)

True

In [ ]:
importlib.reload(utils_MAV)
utils_MAV.mav_set_mode(master, "RTL")

[COMMAND SET MODE] Mode changed > RTL


True

In [7]:
importlib.reload(utils_MAV)
utils_MAV.mav_set_mode(master, "GUIDED")
utils_MAV.mav_arm(master)
utils_MAV.mav_takeoff(master)
# time.sleep(7)
# utils_MAV.mav_reposition(master, 45.4389466, -0.4283327, 8)

True

In [19]:
importlib.reload(utils_MAV)
utils_MAV.mav_reposition(master)

KeyboardInterrupt: 